In [1]:
%matplotlib notebook
import os
import os.path as path
import numpy as np
from scipy.optimize import curve_fit
from faker import Factory
from scipy import interpolate
from sklearn.preprocessing import MinMaxScaler
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import math 
from ipywidgets import interact, interactive, fixed, interact_manual, widgets
import h5py

from pyccapt.calibration.calibration_tools import tools, data_tools, variables, tof_calibration, data_loadcrop
from pyccapt.calibration.calibration_tools import widgets as wd


In [2]:
number_sample = 300
save_fig=False
plot_fig=True 

In [3]:
# flightPathLength = 110 # mm 
# # The initial value for t_0
# t0 = 51.74 # ns

# dataset name
# AL_data_b
# OLO_AL_6_data
# OLO_W_6_data
# OLO_Ni_8_data
# X6Cr17_2V30Min_5_data
# data_1029_Jun-27-2022_15-53_Pd
# data_115_Jul-27-2022_17-44_Powersweep3

In [4]:
tdc, pulse_mode, dataset, flightPathLength_d, t0_d = wd.dataset_tdc_selection()
display(tdc, pulse_mode, dataset, flightPathLength_d, t0_d)

Dropdown(description='TDC model:', options=('surface_concept', 'roentdec'), value='surface_concept')

Dropdown(description='Pulse mode:', options=('voltage', 'laser'), value='voltage')

Text(value='OLO_AL_6_data', description='Dataset:', placeholder='Paste ticket description here!')

Text(value='110', description='Flight path length:', placeholder='Flight path length')

Text(value='51.74', description='t0:', placeholder='T_0 of the instrument')

$$\textbf{You can specify which dataset to use in below block}$$

In [5]:
tdc_model = tdc.value
pulse_mode_ini = pulse_mode.value

dataset_name = dataset.value

flightPathLength = float(flightPathLength_d.value) # mm 
# The initial value for t_0
t0 = float(t0_d.value) # ns

p = path.abspath(path.join("", "../../../.."))

variables.init()

path_main = os.path.join(p, 'tests//data')
filename_main = path_main + '//' + dataset_name + '.h5'

# variables.path = os.path.join(p, 'tests//results//mc_vol_bowl_calibratin')
variables.path = os.path.join(p, 'tests//results//tof_calibration')
variables.result_path = os.path.join(p, 'tests/results/ion_type_selection/' + dataset_name)
if not os.path.isdir(variables.result_path):
        os.makedirs(variables.result_path, mode=0o777, exist_ok=True)
        
filename = variables.path + '//' + dataset_name + '//' + dataset_name + '.h5'



head, tail = os.path.split(filename)
figname = os.path.splitext(tail)[0]

data = data_tools.read_hdf5_through_pandas(filename)


In [6]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da)
0,5216.226562,5216.226562,1572.0,633.384306,-0.031837,0.859592,0.0,1.0,27.713360
1,5216.226562,5216.226562,1791.0,628.981470,-0.350204,-18.720000,219.0,1.0,27.003132
2,5216.226562,5216.226562,1992.0,635.510286,-11.906939,2.610612,201.0,1.0,27.493455
3,5216.226562,5216.226562,2429.0,624.222018,7.195102,-0.827755,437.0,1.0,26.965459
4,5216.226562,5216.226562,3220.0,629.276364,14.644898,7.449796,791.0,1.0,27.027015
...,...,...,...,...,...,...,...,...,...
1858635,5711.025879,5711.025879,3626.0,601.419168,-4.998367,-1.846531,152.0,1.0,26.985305
1858636,5711.025879,5711.025879,3641.0,602.674182,-0.827755,-14.867755,15.0,1.0,26.971058
1858637,5711.025879,5711.025879,4538.0,600.644214,-6.271837,3.565714,897.0,1.0,26.801420
1858638,5711.025879,5711.025879,5881.0,459.993492,34.097143,5.380408,1343.0,1.0,13.680422


In [7]:
mc = data['mc (Da)'].to_numpy()

In [8]:
peaks_sides_p = np.zeros(0)
def mc_his(bin_size, prominence, distance, percent, plot, figname, mc, mc_ideal):

    bin_size = float(bin_size)
    prominence = int(prominence)
    distance = int(distance)
    percent = int(percent)
    if mc_ideal == True:
        mc_ideal = np.array(variables.listMaterial)
        print('List of mc are:', mc_ideal)
    else:
        mc_ideal = np.zeros(0)
    max_hist, left_right_peaks, peaks_sides, max_paek_edges, index_max = tools.massSpecPlot(mc[mc < 100], bin_size, mc_ideal, distance=distance, percent=percent, prominence=prominence, plot=plot, label='tof', fig_name=figname)
    mrp = (max_hist / (left_right_peaks[1] - left_right_peaks[0]))
    print('Mass resolving power for the highest peak (MRP --> m/m_2-m_1):', mrp)
    for i in range(len(peaks_sides)):
        print('Peaks ', i, 'is: {:.2f}'.format(peaks_sides[i,0]), 'peak window sides are: {:.2f} - {:.2f}'.format(peaks_sides[i,2], peaks_sides[i,3]))
    global peaks_sides_p
    peaks_sides_p = peaks_sides

In [9]:
peaks_sides = interact_manual(mc_his, bin_size=widgets.Textarea(value="0.1")
               ,prominence=widgets.Textarea(value="60"), distance=widgets.Textarea(value="100")
               ,percent=widgets.Textarea(value="20"), plot=fixed(True), mc=fixed(mc),  figname=fixed(figname)
                , mc_ideal=[('False',False),('True',True)]);



interactive(children=(Textarea(value='0.1', description='bin_size'), Textarea(value='60', description='promine…

In [10]:
isotopeTableFile = '../../../files/isotopeTable.h5'
dataframe = data_tools.read_hdf5_through_pandas(isotopeTableFile)
elementsList = dataframe['element']
elementIsotopeList = dataframe['isotope']
elementMassList =  dataframe['weight']
abundanceList = dataframe['abundance']

elements = list(zip(elementsList, elementIsotopeList, elementMassList, abundanceList))
dropdownList = []
for element in elements:
    tupleElement = ("{} ({}) ({:.2f})".format(element[0],element[1],element[3]), "{}{}".format(element[0],element[1]))
    dropdownList.append(tupleElement)

chargeList = [(1,1,),(2,2,),(3,3,),(4,4,)]
dropdown = wd.dropdownWidget(dropdownList, "Elements")
dropdown.observe(wd.on_change_ions_selection)


chargeDropdown = wd.dropdownWidget(chargeList, "Charge")
chargeDropdown.observe(wd.on_change_charge_ions_selection)

wd.compute_element_isotope_values_according_to_selected_charge(mode='ions_selection')

buttonAdd = wd.buttonWidget("ADD")
buttonDelete = wd.buttonWidget("DELETE")
buttonReset = wd.buttonWidget("RESET")

display(dropdown)
display(chargeDropdown)
display(buttonAdd)
display(buttonDelete)
display(buttonReset)
listMaterial = buttonAdd.on_click(wd.onClickAdd)
buttonDelete.on_click(wd.onClickDelete)
buttonReset.on_click(wd.onClickReset)

Dropdown(description='Elements', options=(('H (1) (99.98)', 'H1'), ('H (2) (0.01)', 'H2'), ('He (3) (0.00)', '…

Dropdown(description='Charge', options=((1, 1), (2, 2), (3, 3), (4, 4)), value=1)

Button(description='ADD', icon='check', style=ButtonStyle(), tooltip='ADD')

Button(description='DELETE', icon='check', style=ButtonStyle(), tooltip='DELETE')

Button(description='RESET', icon='check', style=ButtonStyle(), tooltip='RESET')

In [11]:
fake = Factory.create()
color = np.zeros(len(mc), dtype='object' )
element = np.zeros(len(mc), dtype='object')

for index, elemen in enumerate(variables.listMaterial):
    mask = np.logical_and((peaks_sides_p[index,2] < mc), (mc < peaks_sides_p[index,3])).squeeze()
    print(mask[mask==True].shape)

    index_true = np.where(mask==True)
    index_get_to_false = np.random.choice(index_true[0], size=int(index_true[0].shape[0] - index_true[0].shape[0]), replace=False)
    mask[index_get_to_false] = False
    print('ploted ions', mask[mask==True].shape)
    
    color[mask] = fake.hex_color()
    element[mask] = elemen



(1047,)
ploted ions (1047,)
(128473,)
ploted ions (128473,)
(1441888,)
ploted ions (1441888,)


In [12]:
data['element'] = element
data['color'] = color

In [13]:
data

,high_voltage (V),pulse (V),start_counter,t (ns),x (mm),y (mm),pulse_pi,ion_pp,mc (Da),element,color
0,5216.226562,5216.226562,1572.0,633.384306,-0.031837,0.859592,0.0,1.0,27.713360,0,0
1,5216.226562,5216.226562,1791.0,628.981470,-0.350204,-18.720000,219.0,1.0,27.003132,Al27(1+),#2c0359
2,5216.226562,5216.226562,1992.0,635.510286,-11.906939,2.610612,201.0,1.0,27.493455,0,0
3,5216.226562,5216.226562,2429.0,624.222018,7.195102,-0.827755,437.0,1.0,26.965459,Al27(1+),#2c0359
4,5216.226562,5216.226562,3220.0,629.276364,14.644898,7.449796,791.0,1.0,27.027015,Al27(1+),#2c0359
...,...,...,...,...,...,...,...,...,...,...,...
1858635,5711.025879,5711.025879,3626.0,601.419168,-4.998367,-1.846531,152.0,1.0,26.985305,Al27(1+),#2c0359
1858636,5711.025879,5711.025879,3641.0,602.674182,-0.827755,-14.867755,15.0,1.0,26.971058,Al27(1+),#2c0359
1858637,5711.025879,5711.025879,4538.0,600.644214,-6.271837,3.565714,897.0,1.0,26.801420,Al27(1+),#2c0359
1858638,5711.025879,5711.025879,5881.0,459.993492,34.097143,5.380408,1343.0,1.0,13.680422,0,0


In [14]:
# save the new data
name_save_file = variables.result_path + '//' + dataset_name + '.h5'
data_tools.store_df_to_hdf(name_save_file, data, 'df')

d:\pyccapt\pyccapt\calibration\calibration_tools\data_tools.py:91: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed-integer,key->block1_values] [items->Index(['element', 'color'], dtype='object')]

  dataframe.to_hdf(filename, key, mode='w')


In [15]:
data.dtypes

high_voltage (V)    float64
pulse (V)           float64
start_counter       float64
t (ns)              float64
x (mm)              float64
y (mm)              float64
pulse_pi            float64
ion_pp              float64
mc (Da)             float64
element              object
color                object
dtype: object